In [1]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'my_column': ['A;B', 'B;C', 'A', 'C;D', 'B', 'M;N', 'O;P', 'A;M']
})

# Split the column by ';' into lists
df['my_column_split'] = df['my_column'].str.split(';')

# Explode the lists into rows
exploded = df.explode('my_column_split')

# Create binary columns for each unique value
dummies = pd.get_dummies(exploded['my_column_split'])

# Group back to original rows by summing
binary_df = dummies.groupby(exploded.index).sum()

# Create business and savings flags
business_flags = {'A', 'B', 'C', 'D', 'E', 'X', 'F'}
savings_flags = {'M', 'N', 'O', 'P'}

binary_df['business'] = binary_df[list(business_flags & set(binary_df.columns))].any(axis=1).astype(int)
binary_df['savings'] = binary_df[list(savings_flags & set(binary_df.columns))].any(axis=1).astype(int)

# Concatenate with original DataFrame if needed
result = pd.concat([df, binary_df], axis=1)

result

,my_column,my_column_split,A,B,C,D,M,N,O,P,business,savings
0,A;B,"[A, B]",1,1,0,0,0,0,0,0,1,0
1,B;C,"[B, C]",0,1,1,0,0,0,0,0,1,0
2,A,[A],1,0,0,0,0,0,0,0,1,0
3,C;D,"[C, D]",0,0,1,1,0,0,0,0,1,0
4,B,[B],0,1,0,0,0,0,0,0,1,0
5,M;N,"[M, N]",0,0,0,0,1,1,0,0,0,1
6,O;P,"[O, P]",0,0,0,0,0,0,1,1,0,1
7,A;M,"[A, M]",1,0,0,0,1,0,0,0,1,1
